# **2986. Find Third Transaction**

**Table: Transactions**
``` console
+------------------+----------+
| Column Name      | Type     |
+------------------+----------+
| user_id          | int      |
| spend            | decimal  |
| transaction_date | datetime |
+------------------+----------+
```
(user_id, transaction_date) is column of unique values for this table.

This table contains user_id, spend, and transaction_date.

Write a solution to find the third transaction (if they have at least three transactions) of every user, where the spending on the preceding two transactions is lower than the spending on the third transaction.

Return the result table by user_id in ascending order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**Transactions table:**
``` console
+---------+--------+---------------------+
| user_id | spend  | transaction_date    | 
+---------+--------+---------------------+
| 1       | 65.56  | 2023-11-18 13:49:42 | 
| 1       | 96.0   | 2023-11-30 02:47:26 |     
| 1       | 7.44   | 2023-11-02 12:15:23 | 
| 1       | 49.78  | 2023-11-12 00:13:46 | 
| 2       | 40.89  | 2023-11-21 04:39:15 |  
| 2       | 100.44 | 2023-11-20 07:39:34 | 
| 3       | 37.33  | 2023-11-03 06:22:02 | 
| 3       | 13.89  | 2023-11-11 16:00:14 | 
| 3       | 7.0    | 2023-11-29 22:32:36 | 
+---------+--------+---------------------+
**Output**
``` console
+---------+-------------------------+------------------------+
| user_id | third_transaction_spend | third_transaction_date | 
+---------+-------------------------+------------------------+
| 1       | 65.56                   | 2023-11-18 13:49:42    |  
+---------+-------------------------+------------------------+
```
**Explanation**
- For user_id 1, their third transaction occurred on 2023-11-18 at 13:49:42 with an amount of $65.56, surpassing the expenditures of the previous two transactions which were $7.44 on 2023-11-02 at 12:15:23 and $49.78 on 2023-11-12 at 00:13:46. Thus, this third transaction will be included in the output table.
- user_id 2 only has a total of 2 transactions, so there isn't a third transaction to consider.
- For user_id 3, the amount of $7.0 for their third transaction is less than that of the preceding two transactions, so it won't be included.
Output table is ordered by user_id in ascending order.

**Solution-1:**

``` sql
# Write your MySQL query statement below

with cte as(
    select *, count(*) OVER(partition by user_id) as cnt,
    rank() over (partition by user_id order by transaction_date) as rn
    from Transactions
),
cte2 as(
    select user_id, spend, transaction_date
    from cte where cnt >= 3 and rn <= 3
)
select distinct
t3.user_id, t3.spend as third_transaction_spend, 
t3.transaction_date as third_transaction_date 
from cte2 t1, cte2 t2, cte2 t3
where t1.user_id = t2.user_id and t1.user_id = t3.user_id
and t1.transaction_date < t2.transaction_date and t2.transaction_date < t3.transaction_date
and t1.transaction_date < t3.transaction_date
and t1.spend < t3.spend and t2.spend < t3.spend 
order by t3.user_id ASC

**Solution-2:**

``` sql
# Write your MySQL query statement below

WITH get_prior_spend AS (
    SELECT
        *
        , RANK() OVER (PARTITION BY user_id ORDER BY transaction_date) AS rk
        , LAG(spend) OVER (PARTITION BY user_id ORDER BY transaction_date) AS second_transaction_spend
        , LAG(spend, 2) OVER (PARTITION BY user_id ORDER BY transaction_date) AS first_transaction_spend
    FROM Transactions
)
SELECT user_id, spend AS third_transaction_spend, transaction_date AS third_transaction_date
FROM get_prior_spend
WHERE
    rk = 3
    AND spend > second_transaction_spend
    AND spend > first_transaction_spend
ORDER BY 1